In [1]:
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
# choices_set_1 = pd.read_excel(io=param_dir + "set1_PerryandCo.xlsx", sheet_name=0)
# response_set_1 = pd.read_excel(io=param_dir + "Perry_and_Co_Responses_set_1.xlsx", sheet_name=0)

In [4]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choises_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names


In [5]:
print (choises_set_1_sheet_names)
print (response_set_1_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']
['S1 - F6', 'S1 - F5', 'S1 - F4', 'S1 - F3', 'S1 - F2', 'S1 - F1']


In [6]:
a_choice_sheet = choices_set_1_xl.parse(choises_set_1_sheet_names[5])
sample_response = response_set_1_xl.parse(response_set_1_sheet_names[5])

In [7]:
print (list(a_choice_sheet.columns))

['Question_in_set', 'Question_overall', 'ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude', 'longitude', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 'ExctAcr', 'LstSrvD', 'county']


## Count number of questions

In [8]:
question_count = 0

for a_choice_sheet in choises_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


In [9]:
# unwanted_opinions = ["kirtir@wsu.edu"]

In [10]:
sample_response.head(2)

,Timestamp,Email Address,"QUESTION 1 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/45.99256829,-119.34481065/@45.99256829,-119.34481065,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?,Comments,"QUESTION 2 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.07535056,-119.65017789/@46.07535056,-119.65017789,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.1,Comments.1,"QUESTION 3 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/45.96712978,-119.37401358/@45.96712978,-119.37401358,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.2,...,Comments.46,"QUESTION 48 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/47.24441649,-119.80872437/@47.24441649,-119.80872437,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.47,Comments.47,"QUESTION 49 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.29637591,-118.8213993/@46.29637591,-118.8213993,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.48,Comments.48,"QUESTION 50 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.39197279,-119.23327008/@46.39197279,-119.23327008,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.49,Comments.49
0,2022-02-02 06:36:57.993,kirtir@wsu.edu,Single Crop,Check for Yes,Weeds or volunteers with herbicide given timi...,Single Crop,NaN,NaN,Single Crop,Check for Yes,...,NaN,Unsure,Check for Yes,NaN,Unsure,Check for Yes,NaN,Single Crop,NaN,NaN
1,2022-02-02 11:07:30.168,andrew.mcguire@wsu.edu,Unsure,Check for Yes,NaN,Single Crop,NaN,NaN,Single Crop,NaN,...,NaN,Single Crop,NaN,NaN,Double Crop,NaN,NaN,Single Crop,NaN,NaN


In [11]:
## Define the damn output dataframe

output_df = pd.DataFrame(columns=['Form', 'Question', 'ID', "Perry", "Andrew", "Tim", "Kirti"], 
                         index=range(question_count))
output_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choise_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choise_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    no_people_in_sheet = a_response_sheet.shape[0]
    
    extended_choices = pd.concat([extended_choices, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if ("Comments" in a_col_name) and (a_response_sheet[a_col_name].isna().sum()<no_people_in_sheet):
            if a_col_name.split(".")[-1].isdigit():
                question_number = str(int(a_col_name.split(".")[-1])+1)
            else:
                question_number = "1"
           
            output_df.loc[curr_row, "Form"] = sheet_numeric_part
            output_df.loc[curr_row, "Question"] = question_number
#            output_df.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            output_df.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
            output_df.loc[curr_row, "Perry"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            output_df.loc[curr_row, "Andrew"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            output_df.loc[curr_row, "Tim"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet["Email Address"]):
                output_df.loc[curr_row, "Kirti"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
# drop rows where all row is NaNs
output_df = output_df.dropna(axis=0, how='all')

In [12]:
output_df.shape

(34, 7)

In [13]:
extended_output = pd.merge(output_df, extended_choices, on=['ID'], how='left')
extended_output.head(2)

,Form,Question,ID,Perry,Andrew,Tim,Kirti,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,6,25,51838_WSDA_SF_2016,NaN,NaN,"Looks too late for mustard, but the sharp decr...",NaN,25,275,AdamBenton2016_51838_WSDA_SF_2016.png,...,51838_WSDA_SF_2016_46.90784511_-119.0583815_TO...,46.907845,-119.058381,bluegrass seed,center pivot,wsda,185,185.294850,2016/08/17 00:00:00,Adams
1,6,31,55524_WSDA_SF_2016,I would say the late curve seen a lot with BG ...,NaN,NaN,NaN,31,281,AdamBenton2016_55524_WSDA_SF_2016.png,...,55524_WSDA_SF_2016_46.75461754_-118.92543835_T...,46.754618,-118.925438,bluegrass seed,center pivot,wsda,141,140.849153,2016/08/03 00:00:00,Adams


In [14]:
# check of the question numbers are correct.
sum(extended_output['Question'].astype(int) - extended_output['Question_in_set'])

0

In [15]:
### Replace NaN with "No"

In [16]:
extended_output = extended_output.fillna("No")

In [17]:
extended_output.head(3)

,Form,Question,ID,Perry,Andrew,Tim,Kirti,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,6,25,51838_WSDA_SF_2016,No,No,"Looks too late for mustard, but the sharp decr...",No,25,275,AdamBenton2016_51838_WSDA_SF_2016.png,...,51838_WSDA_SF_2016_46.90784511_-119.0583815_TO...,46.907845,-119.058381,bluegrass seed,center pivot,wsda,185,185.294850,2016/08/17 00:00:00,Adams
1,6,31,55524_WSDA_SF_2016,I would say the late curve seen a lot with BG ...,No,No,No,31,281,AdamBenton2016_55524_WSDA_SF_2016.png,...,55524_WSDA_SF_2016_46.75461754_-118.92543835_T...,46.754618,-118.925438,bluegrass seed,center pivot,wsda,141,140.849153,2016/08/03 00:00:00,Adams
2,6,35,108682_WSDA_SF_2017,Grass hay will usually be singe crop (except w...,No,No,No,35,288,Grant2017_108682_WSDA_SF_2017.png,...,108682_WSDA_SF_2017_47.06385605_-119.92398299_...,47.063856,-119.923983,grass hay,center pivot,wsda,66,66.189213,2017/07/18,Grant


### Create a long PDF of fields where discussion is requested

In [18]:
import subprocess
os.getcwd()

'/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/Perry_and_Co'

In [19]:
os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_1_PDFs/')

extended_output.sort_values(by=['CropTyp'],
                           ascending = [True],
                           inplace=True)
extended_output.reset_index(inplace=True, drop=True)

with open("set_1_comments.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 1 - Comments}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    file.write("\\section{Comments}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in range(extended_output.shape[0]):
        curr_row = extended_output.loc[a_row]
        TS_file_name = curr_row["NDVI_TS_Name"]
        file.write("\\begin{figure*}[ht]\n")
        file.write("\\centering\n")
        file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                     "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                         TS_file_name + "}\n")
        file.write("\\caption[]{" + \
                   "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                   ", ID: " + curr_row["ID"].replace("_", "\_") + \
                   ", Form: " + curr_row["Form"] + \
                   ", Question: " + curr_row["Question"] + "." + \
                   "}\n")
        
        file.write("\\label{fig:figure" + str(a_row) + "}\n")
        file.write("\\end{figure*}\n")
        file.write("\n")
        
        file.write("\\noindent Perry: " + str(curr_row["Perry"]) + "\\\\\n")
        file.write("Andrew: " + str(curr_row["Andrew"]) + "\\\\\n")
        
        file.write("Tim: " + str(curr_row["Tim"]) +  "\\\\\n")
        file.write("Kirti: " + str(curr_row["Kirti"]) + "\\\\\n")
        
        
        file.write("\\clearpage\n")
        
        file.write("\n")
        file.write("\n")
        file.write("\n")

    file.write("\\end{document}\n")

In [20]:
# extended_output[extended_output.Kirti != "No"]

In [21]:
# extended_output[extended_output.Kirti == "No"]